In [1]:
!pip install fastapi uvicorn pyngrok pandas prophet --quiet


In [2]:
from pyngrok import ngrok
#ngrok.set_auth_token("380ZzaoDdXGvfjMya7VVYPbpJA4_3dcMEvWNppZQDdUGs9j7p")
# 2vgULFP5rjcVsMkwGpjJUGvyct2_9nT4An12iTiYqTZStQPf(Rashmita auth token)
ngrok.set_auth_token("384j7T1eOSSQkhHBCRhr9dJF7V7_4hCghtg8bCFqjvWUEtN6P")      #suhani's

In [3]:
%%writefile backend.py
from fastapi import FastAPI, UploadFile, File
from fastapi.responses import JSONResponse
import pandas as pd
import numpy as np
import traceback

from prophet import Prophet
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN


app = FastAPI(title="FitPulse Backend")

# ---------------- GLOBAL STATE ----------------
CLEAN_DF = None
FEATURE_DF = None
ALERTS_DF = None

@app.get("/")
def root():
    return {"message": "Backend is running"}

# ==================================================
# MODULE 1 – PREPROCESSING
# ==================================================
@app.post("/preprocess")
async def upload_csv(file: UploadFile = File(...)):
    global CLEAN_DF

    df = pd.read_csv(file.file)

    REQUIRED_COLUMNS = [
        "user_id",
        "date",
        "TotalSteps",
        "avg_heart_rate",
        "total_sleep_minutes"
    ]   # Check required columns

    missing_cols = [col for col in REQUIRED_COLUMNS if col not in df.columns]

    if missing_cols:
        return JSONResponse(
            status_code=400,
            content={"error": f"Missing required columns: {missing_cols}"}
        )

    df["date"] = pd.to_datetime(df["date"], errors="coerce")  # Convert date column to datetime
    df = df.dropna(subset=["date"])  # Drop rows where date conversion failed

    NUMERIC_COLS = [
        "TotalSteps",
        "avg_heart_rate",
        "total_sleep_minutes"
    ]  # Convert important columns to numeric

    for col in NUMERIC_COLS:
        df[col] = pd.to_numeric(df[col], errors="coerce")

    # Handle missing values
    df["TotalSteps"].fillna(0, inplace=True)
    df["avg_heart_rate"].fillna(df["avg_heart_rate"].median(), inplace=True)
    df["total_sleep_minutes"].fillna(df["total_sleep_minutes"].median(), inplace=True)

    df = (
        df.groupby(["user_id", "date"], as_index=False)
          .agg({
              "TotalSteps": "sum",
              "avg_heart_rate": "mean",
              "total_sleep_minutes": "mean"
          })
    )       # Daily aggregation per user

    CLEAN_DF = df
    df.to_csv("clean_data.csv", index=False)  # Save cleaned data

    return {
        "status": "success",
        "rows": len(df)
    }  # Return response


Writing backend.py


In [4]:
import threading
import uvicorn

def run_server():
    uvicorn.run("backend:app", host="0.0.0.0", port=8000)

threading.Thread(target=run_server).start()


In [6]:
public_url = ngrok.connect(8000)
print("PUBLIC URL:", public_url)


PUBLIC URL: NgrokTunnel: "https://renowned-zuri-coleopterous.ngrok-free.dev" -> "http://localhost:8000"
